In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')
train.rename(columns={'index': 'id'}, inplace=True)
test = pd.read_csv('test.csv')
sample_submit = pd.read_csv('sample_submit.csv', names=['id', 'Y'])

In [3]:
data = pd.concat([train, test], sort=False)

In [4]:
# あまり効果なし
# print(data['education'].unique())
# data.loc[data['education'] == 'Some-college', 'education'] = 'Bachelors'
# data.loc[data['education'] == '12th', 'education'] = 'junior-glad'
# data.loc[data['education'] == '10th', 'education'] = 'junior-glad'
# data.loc[data['education'] == '5th-6th', 'education'] = 'junior-glad'
# data.loc[data['education'] == '7th-8th', 'education'] = 'junior-glad'
# data.loc[data['education'] == '11th', 'education'] = 'junior-glad'
# data.loc[data['education'] == '9th', 'education'] = 'junior-glad'
# data.loc[data['education'] == '1st-4th', 'education'] = 'junior-glad'


# print(data['education'].unique())
# data


In [5]:
data['workclass'].unique()

array(['Private', '?', 'Local-gov', 'Self-emp-inc', 'Federal-gov',
       'Self-emp-not-inc', 'State-gov'], dtype=object)

In [6]:
# カテゴリカルな特徴量は全てとりあえずエンコーディングしておく．
categorical_column = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
for column in categorical_column:
    d = {}
    for i, category in enumerate(data[column].unique()):
        d[category] = i
    data[column] = data[column].map(d)
data

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,native-country,Y
0,0,22,0,132618,0,12,0,0,0,0,0,0,0.0
1,1,22,0,132655,1,9,1,1,1,0,1,1,1.0
2,2,23,1,132674,2,8,0,2,2,0,0,0,0.0
3,3,36,0,132642,1,13,1,3,1,0,1,0,1.0
4,4,49,0,132646,3,13,1,4,1,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6795,16995,31,0,132530,0,9,2,8,2,0,0,0,NaN
6796,16996,31,0,132634,1,6,0,12,2,1,1,0,NaN
6797,16997,31,0,132598,2,13,4,1,2,0,1,0,NaN
6798,16998,23,0,132573,0,8,2,10,2,2,0,0,NaN


In [7]:
data.drop('id', axis=1, inplace=True)
# data.drop('fnlwgt', axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Y']
X_train = train.drop('Y', axis=1)
X_test = test.drop('Y', axis=1)
X_train


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,native-country
0,22,0,132618,0,12,0,0,0,0,0,0
1,22,0,132655,1,9,1,1,1,0,1,1
2,23,1,132674,2,8,0,2,2,0,0,0
3,36,0,132642,1,13,1,3,1,0,1,0
4,49,0,132646,3,13,1,4,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
10195,31,0,132650,1,9,2,9,5,0,0,1
10196,36,0,132590,2,9,1,4,3,0,1,1
10197,36,2,132498,1,13,1,10,1,2,1,0
10198,17,0,132614,0,9,0,10,2,0,0,0


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3,
                     random_state=0, stratify=y_train)

In [9]:
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb


y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.iloc[train_index, :]
    X_val = X_train.iloc[valid_index, :]
    y_tr = y_train.iloc[train_index]
    y_val = y_train.iloc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    model = lgb.train(params, lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      verbose_eval=10,
                      num_boost_round=1000,
                      early_stopping_rounds=10)

    oof_train[valid_index] = model.predict(
        X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)


/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yutahirai/opt/anaconda3/envs/matlab/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1384, number of negative: 4328
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000633 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 411
[LightGBM] [Info] Number of data points in the train set: 5712, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242297 -> initscore=-1.140128
[LightGBM] [Info] Start training from score -1.140128
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.384523	valid_1's binary_logloss: 0.384879
[20]	training's binary_logloss: 0.312615	valid_1's binary_logloss: 0.315186
[30]	training's binary_logloss: 0.273892	valid_1's binary_logloss: 0.280893
[40]	training's binary_logloss: 0.249787	valid_1's binary_logloss: 0.26289
[50]	training's binary_logloss: 0.232933	valid_1's binary_logloss: 0.253133
[60

In [10]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)


===CV scores===
[0.24625620168082912, 0.27368164424727387, 0.24993106000326373, 0.2903278434376668, 0.26862417864564886]
0.26576418560293646


In [11]:
from sklearn.metrics import accuracy_score


y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

0.8743697478991597

In [12]:
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

array([0, 0, 0, 0, 1, 1, 0, 1, 1, 0])

In [13]:
lgb.plot_tree(model, figsize=(200,300), orientation='vertical')

<AxesSubplot:>

In [14]:
sample_submit['Y'] = y_sub
sample_submit.to_csv('submit-lightgbm.csv', header=False, index=False)
sample_submit

,id,Y
0,10200,0
1,10201,0
2,10202,0
3,10203,0
4,10204,1
...,...,...
6795,16995,0
6796,16996,0
6797,16997,0
6798,16998,0


In [15]:
# y_pred = model.predict(X_train, num_iteration=model.best_iteration)
# x_ = np.arange(100)/100.
# y_ = []
# for i in np.arange(100)/100.:
#     y_.append(accuracy_score(y_train, (y_pred > i).astype(int)))
# plt.plot(x_,y_)
# print(y_[np.argmax(y_)])
# print(x_[np.argmax(y_)])

In [16]:
# y_pred = model.predict(X_test, num_iteration=model.best_iteration)
# y_pred = (y_pred > x_[np.argmax(y_)]).astype(int)
# sample_submit['Y'] = y_pred
# sample_submit.to_csv('submit-lightgbm.csv', header=False, index=False)
# sample_submit